# 财务数据与因子分析策略2-多因子策略介绍

## 简单因子相关性

* classwork 1

把上节课，简单多因子分析方法所有代码简化并整理到一起。然后修改参数尝试分析其他财务因子。

In [1]:
from scipy.stats.mstats import winsorize
from scipy.stats import pearsonr,spearmanr,kendalltau
import pandas as pd
import numpy as np

from gm.api import *
set_token("eea4b28a3ceb7048603d388bb777ebc67de47385")

In [4]:
def standardize_series(series):#标准化
    std = series.std()
    mean = series.mean()
    return (series-mean)/std

index_s='SHSE.000300'
table_s='deriv_finance_indicator'
#'trading_derivative_indicator'
start_date_s='2016-01-01'
end_date_s='2016-06-01'
fields_s='ROEANNUAL,NPGRT,OPNCFPS'
#'TCLOSE,NEGOTIABLEMV,TOTMKTCAP,TURNRATE,PELFY,PETTM,PEMRQ,PELFYNPAAEI,PETTMNPAAEI'

hs300=get_constituents(index=index_s, fields='symbol, weight', df=True)
ex2=get_fundamentals(table=table_s, symbols=hs300.symbol.tolist(), start_date=start_date_s, end_date=end_date_s,
                 fields=fields_s, df=True,limit=40000)
ex3=ex2.set_index(["end_date","symbol"])

history_data=history(symbol=hs300.symbol.tolist(), frequency='1d', start_time=start_date_s,  end_time=end_date_s, fields='symbol,close,eob', adjust=ADJUST_PREV, df= True)
history_data_n=history_data.set_index(["eob","symbol"]).unstack().close
history_data_p=history_data_n/history_data_n.shift(1)-1

h_m=history_data_p.shift(-1).stack()

col=[]
for i in ex3.columns[1:]:
    z1=winsorize(ex3[i],limits=0.03)
    col.append(i+"s")
    ex3[i+"s"]=standardize_series(z1)
    
    
f_y_1=pd.concat([h_m,ex3],axis=1).dropna()

for i in col:
    print(i,spearmanr(f_y_1.iloc[:,0],f_y_1[i]))


ROEANNUALs SpearmanrResult(correlation=-0.06365697164615212, pvalue=0.3633436720873816)
NPGRTs SpearmanrResult(correlation=-0.03908868572571857, pvalue=0.5769621153584988)
OPNCFPSs SpearmanrResult(correlation=0.11837676919698044, pvalue=0.09014156644234972)


## 单因子策略框架


### classwork 2
* 小市值策略“TOTMKTCAP”
* 策略运行时间2013-01-01到2014-12-30

https://www.bilibili.com/video/BV1uq4y177uX/

In [10]:
input()

1


'1'

In [2]:
from datetime import timedelta

In [48]:
# 获取筛选时间：date1表示当前日期之前的100天，date2表示当前时间
date1 = (context.now - timedelta(days=100)).strftime("%Y-%m-%d %H:%M:%S")
date2 = context.now.strftime("%Y-%m-%d %H:%M:%S")

# 通过get_instruments获取所有的上市股票代码
all_stock = get_instruments(exchanges='SHSE, SZSE', sec_types=[1], fields='symbol, listed_date, delisted_date',
                            df=True)

# 剔除停牌和st股和上市不足100日的新股和退市股和B股
code = all_stock[(all_stock['listed_date'] < date1) & (all_stock['delisted_date'] > date2) &
                 (all_stock['symbol'].str[5] != '9') & (all_stock['symbol'].str[5] != '2')]

NameError: name 'context' is not defined

In [7]:
cw_data=get_fundamentals_n(table='trading_derivative_indicator', symbols=hs300.symbol.tolist(), end_date="2016-01-01", count=1,
               fields='TOTMKTCAP',df=True)
to_buy=cw_data.sort_values(by="TOTMKTCAP",ascending=False).head(10).symbol.tolist()

In [9]:
cw_data.sort_values(by="TOTMKTCAP")

,symbol,pub_date,end_date,TOTMKTCAP
55,SZSE.002352,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,3.7370e+09
30,SZSE.002120,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,4.0500e+09
212,SHSE.600760,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,5.3328e+09
174,SZSE.000708,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,5.4873e+09
82,SZSE.002607,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,5.6038e+09
...,...,...,...,...
50,SHSE.601628,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,5.8951e+11
94,SHSE.601988,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,8.4517e+11
5,SHSE.601288,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,9.4980e+11
31,SHSE.601398,2015-12-31 00:00:00+08:00,2015-12-31 00:00:00+08:00,1.2348e+12


## 多因子策略框架

伟文同学的因子：表：'deriv_finance_indicator'   此三因子'ROEANNUAL,NPGRT,OPNCFPS' 最大的

get_fundamentals_n

set_index

sort_values


ROEANNUAL	年化净资产收益率	%	净资产收益率＊12/报告期所占月数

NPGRT	归属母公司净利润增长率

OPNCFPS	每股经营活动产生的现金流量净额

### classwork 3

* 通过get_fundamentals_n获取2020-01-01最近一期的沪深300的'ROEANNUAL,NPGRT,OPNCFPS'，然后分别对此三个因子排序，以排序序号作为加权因子，选择三因子加权求和得到最大的前10只股票

* 在掘金中实现以上策略，时间选择2020-01-01到2021-12-30， 选股周期为1个月

In [4]:
index_s='SHSE.000300'
hs300=get_constituents(index=index_s, fields='symbol, weight', df=True)

In [ ]:
ex2=get_fundamentals(table=table_s, symbols=hs300.symbol.tolist(), start_date=start_date_s, end_date=end_date_s,
                 fields=fields_s, df=True,limit=40000)
ex3=ex2.set_index(["end_date","symbol"])

In [5]:
cw_data=get_fundamentals_n(table='deriv_finance_indicator', symbols=hs300.symbol.tolist(), end_date="2020-01-01", count=1,
                            fields='ROEANNUAL,NPGRT,OPNCFPS',df=True)

In [8]:
cw_data.set_index("symbol",inplace=True)

In [9]:
cw_data

,pub_date,end_date,ROEANNUAL,NPGRT,OPNCFPS
symbol,,,,,
SZSE.002236,2020-04-03 00:00:00+08:00,2019-12-31 00:00:00+08:00,22.5615,26.0422,0.5329
SZSE.002241,2020-04-17 00:00:00+08:00,2019-12-31 00:00:00+08:00,8.1802,47.5751,1.6799
SHSE.601618,2020-04-01 00:00:00+08:00,2019-12-31 00:00:00+08:00,9.4252,3.5805,0.8482
SZSE.002252,2020-04-25 00:00:00+08:00,2019-12-31 00:00:00+08:00,5.4382,140.0352,0.1754
SZSE.300628,2020-04-15 00:00:00+08:00,2019-12-31 00:00:00+08:00,31.2919,45.0767,1.8225
...,...,...,...,...,...
SHSE.600999,2020-03-28 00:00:00+08:00,2019-12-31 00:00:00+08:00,10.7274,64.5741,1.4910
SZSE.002230,2020-04-22 00:00:00+08:00,2019-12-31 00:00:00+08:00,8.4499,51.1211,0.6966
SZSE.300558,2020-04-21 00:00:00+08:00,2019-12-31 00:00:00+08:00,9.9315,38.3670,1.3842


In [12]:
f_r=cw_data.sort_values("ROEANNUAL",ascending=False)

In [19]:
f_r["rs"]=list(range(1,len(f_r)+1))

In [24]:
f_r["rs"]

symbol
SHSE.688169      1
SHSE.600132      2
SZSE.300896      3
SZSE.002607      4
SHSE.603195      5
              ... 
SHSE.688126    296
SZSE.000625    297
SHSE.688561    298
SZSE.002044    299
SZSE.002466    300
Name: rs, Length: 300, dtype: int64

In [25]:
f_n=cw_data.sort_values("NPGRT",ascending=False)
f_n["ns"]=list(range(1,len(f_n)+1))

In [27]:
f_o=cw_data.sort_values("OPNCFPS",ascending=False)
f_o["os"]=list(range(1,len(f_o)+1))

In [29]:
fz=f_r["rs"]+f_n["ns"]+f_o["os"]

In [31]:
fz.sort_values()

symbol
SHSE.688169     31
SHSE.603160     40
SZSE.002714     42
SHSE.600132     57
SHSE.688036     73
              ... 
SHSE.688126    834
SZSE.300142    848
SZSE.002044    849
SHSE.600010    855
SZSE.002709    866
Length: 300, dtype: int64

In [34]:
fz.sort_values().head(10).index.tolist()

['SHSE.688169',
 'SHSE.603160',
 'SZSE.002714',
 'SHSE.600132',
 'SHSE.688036',
 'SZSE.300896',
 'SZSE.300498',
 'SHSE.605499',
 'SZSE.000661',
 'SHSE.600346']

In [20]:
f_r=cw_data.sort_values("ROEANNUAL",ascending=False)
f_r["rs"]=list(range(1,len(f_r)+1))

,pub_date,end_date,ROEANNUAL,NPGRT,OPNCFPS,rs
symbol,,,,,,
SHSE.688169,2020-04-28 00:00:00+08:00,2019-12-31 00:00:00+08:00,71.6893,154.5155,15.1095,1
SHSE.600132,2020-04-18 00:00:00+08:00,2019-12-31 00:00:00+08:00,58.6297,158.1524,4.2360,2
SZSE.300896,2020-06-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,57.7899,148.6836,3.4413,3
SZSE.002607,2020-03-10 00:00:00+08:00,2019-12-31 00:00:00+08:00,56.5202,56.5243,0.4011,4
SHSE.603195,2020-04-27 00:00:00+08:00,2019-12-31 00:00:00+08:00,52.3640,37.3829,4.2543,5
...,...,...,...,...,...,...
SHSE.688126,2020-03-31 00:00:00+08:00,2019-12-31 00:00:00+08:00,-2.1263,-902.3971,0.4767,296
SZSE.000625,2020-04-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,-5.8638,-488.8069,0.8083,297
SHSE.688561,2020-05-11 00:00:00+08:00,2019-12-31 00:00:00+08:00,-10.6215,43.2246,-1.9283,298


,symbol,pub_date,end_date,ROEANNUAL,NPGRT,OPNCFPS
0,SHSE.601966,2020-04-14 00:00:00+08:00,2019-12-31 00:00:00+08:00,16.2787,41.2040,2.3332
1,SZSE.002594,2020-04-22 00:00:00+08:00,2019-12-31 00:00:00+08:00,3.1146,-41.9303,5.4033
2,SHSE.688036,2020-04-28 00:00:00+08:00,2019-12-31 00:00:00+08:00,29.4702,172.7963,5.0506
3,SHSE.603019,2020-03-28 00:00:00+08:00,2019-12-31 00:00:00+08:00,15.1879,37.8603,3.5252
4,SHSE.601985,2020-04-24 00:00:00+08:00,2019-12-31 00:00:00+08:00,9.7984,0.4380,1.7131
...,...,...,...,...,...,...
295,SHSE.688009,2020-03-26 00:00:00+08:00,2019-12-31 00:00:00+08:00,11.8454,11.9502,0.3184
296,SZSE.002568,2020-03-31 00:00:00+08:00,2019-12-31 00:00:00+08:00,15.3137,142.6697,1.0206
297,SZSE.002624,2020-04-24 00:00:00+08:00,2019-12-31 00:00:00+08:00,16.7407,-11.9163,1.5647
298,SHSE.601898,2020-03-21 00:00:00+08:00,2019-12-31 00:00:00+08:00,5.9465,67.8984,1.6578


,pub_date,end_date,ROEANNUAL,NPGRT,OPNCFPS,s1
symbol,,,,,,
SZSE.002466,2020-04-29 00:00:00+08:00,2019-12-31 00:00:00+08:00,-69.9844,-371.9572,1.5943,1
SZSE.002044,2020-04-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,-12.4836,-205.5841,0.3213,2
SHSE.688561,2020-05-11 00:00:00+08:00,2019-12-31 00:00:00+08:00,-10.6215,43.2246,-1.9283,3
SZSE.000625,2020-04-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,-5.8638,-488.8069,0.8083,4
SHSE.688126,2020-03-31 00:00:00+08:00,2019-12-31 00:00:00+08:00,-2.1263,-902.3971,0.4767,5
...,...,...,...,...,...,...
SHSE.603195,2020-04-27 00:00:00+08:00,2019-12-31 00:00:00+08:00,52.3640,37.3829,4.2543,296
SZSE.002607,2020-03-10 00:00:00+08:00,2019-12-31 00:00:00+08:00,56.5202,56.5243,0.4011,297
SZSE.300896,2020-06-30 00:00:00+08:00,2019-12-31 00:00:00+08:00,57.7899,148.6836,3.4413,298


Index(['SHSE.688169', 'SHSE.603160', 'SZSE.002714', 'SHSE.600132',
       'SHSE.688036', 'SZSE.300896', 'SZSE.300498', 'SHSE.605499',
       'SZSE.000661', 'SHSE.600346'],
      dtype='object', name='symbol')